# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [2]:
# Do all imports and installs here
import pandas as pd
import string
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import StringType

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

In [3]:
# Read in the data here
# fname = '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'
# df = pd.read_sas(fname, 'sas7bdat', encoding="ISO-8859-1")

fname = 'immigration_data_sample.csv'
df = pd.read_csv(fname)

In [4]:
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()

In [5]:
#df_spark =spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')
df_spark = spark.read.csv("immigration_data_sample.csv", inferSchema=True, header=True)
df_spark.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- cicid: double (nullable = true)
 |-- i94yr: double (nullable = true)
 |-- i94mon: double (nullable = true)
 |-- i94cit: double (nullable = true)
 |-- i94res: double (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: double (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: double (nullable = true)
 |-- i94bir: double (nullable = true)
 |-- i94visa: double (nullable = true)
 |-- count: double (nullable = true)
 |-- dtadfile: integer (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: double (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: integer (nullable = true)
 |-- airline: string (nullable = 

In [ ]:
#write to parquet
df_spark.write.parquet("sas_data")
df_spark=spark.read.parquet("sas_data")
df_spark.head()

In [6]:
cities_df_spark = spark.read.csv("us-cities-demographics.csv",sep=";", inferSchema=True, header=True)

In [14]:
fname = '../../data2/GlobalLandTemperaturesByCity.csv'
temprature_df_spark = spark.read.csv(fname, inferSchema=True, header=True)

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

In [9]:
# Performing cleaning tasks here

# cleaning process to get the values of encoded data in the dictionary at 'i94port.txt' and 'i94res.txt'
#i94port
with open('i94port.txt', 'r') as i94portData:
    i94portData = i94portData.readlines()
#i94cty
with open('i94res.txt', 'r') as i94resData:
     i94resData = i94resData.readlines()
        
#i94port
i94portData = {i.translate(str.maketrans('\t",', '   ')) for i in i94portData}
i94portData = {i.translate(str.maketrans("'", " ")) for i in i94portData}

#i94res
i94resData = {i.translate(str.maketrans(",'", '  ')) for i in i94resData} 
   
#i94port splitting process
i94portData = {k.split('=')[0].strip():k.split('=')[1].strip() for k in i94portData} 

#i94res splitting process
i94resData = {k.split('=')[0].strip():k.split('=')[1].strip() for k in i94resData} 
    

In [10]:
# transforming the values of 'i94port' and 'i94res' 

#i94port
i94port= df['i94port'].values.tolist()
ports = []
for i in i94port:
    ports.append(i94portData[i])

# convert 'i94res' data type to fit the values of the values in the provided data dictionary in 'i94res.txt'
df['i94res'] = df['i94res'].astype(int)
    
#i94cty
i94res= df['i94res'].values.tolist()
i94res = [str(item) for item in i94res]
    
ctyRes = []
for i in i94res:
    ctyRes.append(i94resData[i])
        
df['i94port'] = ports
df['i94res'] = ctyRes
    
i94DF = df.loc[:, ['cicid','i94port', 'i94res']]
i94DF = i94DF.rename(columns = {'i94port': 'port', 'i94res': 'original_residancy'})
    
i94dfSpark = spark.createDataFrame(i94DF)
    
# append the transformed data into immigration log dataframes
immigration = df_spark.withColumn("arrdate", to_timestamp(col("arrdate")))\
                              .withColumn("arrdate",to_date(col("arrdate")))\
                              .withColumnRenamed('arrdate', 'arrival_date')\
                              .join(i94dfSpark, "cicid", "inner")
immigration_log = immigration.createOrReplaceTempView("immigration_log")


### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [11]:
# Write code here

#process_cities_demographics_data

        #  ---------cities_demographics dimension table ------------

cities_demographics_table = cities_df_spark.select('City', 'State Code', 'State','Median Age', \
                                     'Male Population', 'Female Population', 'Total Population'\
                                      , 'Number of Veterans', 'Foreign-born', 'Race')\
                               .withColumnRenamed('State Code', 'State_Code')\
                               .withColumnRenamed('Median Age', 'Median_Age')\
                               .withColumnRenamed('Male Population', 'Male_Population')\
                               .withColumnRenamed('Female Population', 'Female_Population')\
                               .withColumnRenamed('Total Population', 'Total_Population')\
                               .withColumnRenamed('Number of Veterans', 'Number_of_Veterans')\
                               .withColumn('City', upper(col('City')))\
                               .dropDuplicates()
    
cities_demographics= cities_demographics_table.createOrReplaceTempView("cities_demographics")
    



In [16]:
#process_temprature_data
US_weather = temprature_df_spark.filter(temprature_df_spark.Country == "United States")
    
        #  ---------US_weather dimension table ------------
    
US_weather_table = US_weather.select('dt', 'AverageTemperature','City', 'Latitude', 'Longitude')\
                       .withColumn("dt",to_date(col("dt")))\
                       .withColumn('City', upper(col('City')))\
                       .dropDuplicates()
    
US_weather = US_weather_table.createOrReplaceTempView("US_weather")

In [12]:
#process_immigration_data

 #  ---------i94_card dimension table ------------
    
i94Card_table = immigration.select('cicid','i94yr', 'i94mon', 'depdate', 'i94visa', \
                          'visatype', 'admnum', 'dtaddto')\
                          .withColumnRenamed('cicid', 'immigration_id')\
                          .withColumn("depdate", to_timestamp(col("depdate")))\
                          .withColumn("depdate",to_date(col("depdate")))\
                          .withColumn("dtaddto", to_timestamp(col("dtaddto")))\
                          .withColumn("dtaddto",to_date(col("dtaddto")))\
                          .withColumnRenamed('i94yr', 'i94year')\
                          .withColumnRenamed('i94mon', 'i94month')\
                          .withColumnRenamed('depdate', 'depature_date')\
                          .withColumnRenamed('admnum', 'admission_number')\
                          .withColumnRenamed('dtaddto', 'admission_date')\
                          .dropDuplicates()\
                          .withColumn("i94card_id", monotonically_increasing_id()) 
    
i94Card = i94Card_table.createOrReplaceTempView("i94Card")

In [17]:
 # ----immigrants dimension table ------------

immigrants_table = immigration.select('cicid', 'biryear', 'gender', 'occup','i94visa', 'visatype'\
                                        , 'original_residancy')\
                                    .withColumnRenamed('cicid', 'immigration_id')\
                                    .withColumnRenamed('biryear', 'birth_year')\
                                    .withColumnRenamed('occup', 'occupation')\
                                    .dropDuplicates()\
                                    .withColumn("immigrant_id", monotonically_increasing_id())
        
immigrants = immigrants_table.createOrReplaceTempView("immigrants")

In [19]:
# immigration fact table 

immigration_fact_table = spark.sql('''
              SELECT  
              DISTINCT
              immigration_log.cicid AS immigration_id,
              immigration_log.arrival_date,
              immigration_log.i94mode AS immigration_model,
              immigration_log.port,
              cities_demographics.City,
              immigration_log.i94addr AS residancy_state,
              i94Card.i94card_id,
              immigrants.immigrant_id,
              US_weather.AverageTemperature,
              US_weather.Latitude,
              US_weather.Longitude
              FROM  
                  immigration_log JOIN i94Card ON immigration_log.cicid == i94Card.immigration_id
                  JOIN immigrants ON immigration_log.cicid == immigrants.immigration_id
                  LEFT JOIN cities_demographics ON immigration_log.port LIKE '%' || cities_demographics.City || '%'
                  LEFT JOIN US_weather ON (US_weather.dt == immigration_log.arrival_date AND 
                  US_weather.City == cities_demographics.City)
                   ''')
immigration_fact = immigration_fact_table.createOrReplaceTempView("immigration_fact")
     
#immigration_fact_table.write.partitionBy('residancy_state','City').parquet('immigration_fact/')

In [20]:
immigration_fact_table.show(5)

+--------------+------------+-----------------+--------------+---------+---------------+-------------+-------------+-------------------+--------+---------+
|immigration_id|arrival_date|immigration_model|          port|     City|residancy_state|   i94card_id| immigrant_id| AverageTemperature|Latitude|Longitude|
+--------------+------------+-----------------+--------------+---------+---------------+-------------+-------------+-------------------+--------+---------+
|     3095246.0|  1970-01-01|              1.0|SAN DIEGO   CA|SAN DIEGO|             CA| 429496729603|1262720385025|             13.829|  32.95N|  117.77W|
|      861557.0|  1970-01-01|              1.0|SAN DIEGO   CA|SAN DIEGO|           null|1322849927168|  17179869184|             13.829|  32.95N|  117.77W|
|     3669540.0|  1970-01-01|              1.0|BALTIMORE   MD|BALTIMORE|           null| 369367187456| 858993459200|-3.6489999999999996|  39.38N|   76.99W|
|     5771544.0|  1970-01-01|              1.0|BALTIMORE   MD|BA

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [21]:
# Perform quality checks here
#check the uniqueness of immigration_id in immigration_fact_table, i94Card_table, and immigrants_table
if (immigration_fact_table.select('immigration_id').count() != immigration_fact_table.select('immigration_id').dropDuplicates().count()):
    print("immigration_id at immigration_fact_table are duplicated!")
else:
        print("immigration_id at immigration_table are unique!")
        
if (i94Card_table.select('immigration_id').count() != i94Card_table.select('immigration_id').dropDuplicates().count()):
    print("immigration_id at i94Card_table are duplicated!")
else:
    print("immigration_id at i94Card_table are unique!")
        
if (immigrants_table.select('immigration_id').count() != immigrants_table.select('immigration_id').dropDuplicates().count()):
    print("immigration_id at immigrants_table are duplicated!")
else:
    print("immigration_id at immigrants_table are unique!")
        


immigration_id at immigration_fact_table are duplicated!
immigration_id at i94Card_table are unique!
immigration_id at immigrants_table are unique!


In [ ]:
#check missing values of immigration_id in immigration_table, i94Card_table, and immigrants_table
   
print("The NULL values at immigration_id column of immigration_table are shown below!")
print(immigration_fact_table.select(count(when(col('immigration_id').isNull() , immigration_fact_table.immigration_id))).show())
        
print("The NULL values at immigration_id column of i94Card_table are shown below!")
print(i94Card_table.select(count(when(col('immigration_id').isNull() , i94Card_table.immigration_id))).show())
        
print("The NULL values at immigration_id column of immigrants_table are shown below!")
print(immigrants_table.select(count(when(col('immigration_id').isNull() , immigrants_table.immigration_id))).show())
        

The NULL values at immigration_id column of immigration_table are shown below!


In [ ]:
        
#check that all immigration_ids are transformed appropriatly from immigration_log
if((i94Card_table.select('immigration_id').count() < 1) or (immigration.select('cicid').count() != i94Card_table.select('immigration_id').count())):
    print(str(immigration.select('cicid').count() - i94Card_table.select('immigration_id').count()) + " rows at i94Card_table did not transformed appropriatly from immigration log!")
        
if((immigrants_table.select('immigration_id').count() < 1) or (immigration.select('cicid').count() != immigrants_table.select('immigration_id').count())):
    print(str(immigration.select('cicid').count() - immigrants_table.select('immigration_id').count()) +" rows at i94Card_table did not transformed appropriatly from immigration log!")

if((immigration_fact_table.select('immigration_id').count() < 1) or (immigration.select('cicid').count() != immigration_fact_table.select('immigration_id').count())):
    print(str(immigration.select('cicid').count() - i94Card_table.select('immigration_id').count()) + " rows at i94Card_table did not transformed appropriatly from immigration log!")

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.